In [ ]:
# ==============================================================================
# TREINAMENTO DO DETECTOR DE SPOTLIGHT (MobileNetV3)
# ==============================================================================
# Dataset: SpotBID_Organized_v3 (Validado com Convex Hull)
# Modelo: MobileNetV3 Small (Transfer Learning - Leve e Rápido)
# ==============================================================================

import os
import shutil
import time
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# 1. SETUP E GPU
from google.colab import drive
drive.mount('/content/drive')

# Verificar se a GPU está ligada
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Hardware selecionado: {device}")

if device.type != 'cuda':
    raise RuntimeError("❌ ERRO: GPU não detectada! Vá em Runtime > Change runtime type > T4 GPU")

# --- CAMINHOS (Ajustado para sua pasta v3) ---
# Onde estão os dados organizados no Drive
DRIVE_SOURCE_PATH = Path("/content/drive/MyDrive/meus_modelos_ia/datasets/SpotBID_Organized_v3")

# Caminho temporário no SSD do Colab (Para treinar rápido)
LOCAL_FAST_PATH = Path("/content/spotbid_fast")

# Onde salvar o modelo final treinado
OUTPUT_MODEL_DIR = Path("/content/drive/MyDrive/document-classifier/models/quality_detector")
OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

# ==============================================================================
# 2. CARREGAMENTO ULTRA-RÁPIDO (Cópia Drive -> SSD Local)
# ==============================================================================
def prepare_fast_storage():
    if LOCAL_FAST_PATH.exists():
        print("✅ Dataset já existe no disco local. Pulando cópia.")
        return

    if not DRIVE_SOURCE_PATH.exists():
        raise FileNotFoundError(f"❌ Não encontrei a pasta: {DRIVE_SOURCE_PATH}")

    print("🚀 Copiando dataset do Drive para SSD local (Acelerador de Treino)...")
    print("   Isso vai levar uns 3-5 minutos, mas vale muito a pena!")

    start_time = time.time()
    shutil.copytree(DRIVE_SOURCE_PATH, LOCAL_FAST_PATH)
    end_time = time.time()

    print(f"✅ Cópia concluída em {(end_time - start_time)/60:.1f} minutos!")

# Executar a cópia
prepare_fast_storage()

# ==============================================================================
# 3. DATALOADERS E TRANSFORMAÇÕES
# ==============================================================================
BATCH_SIZE = 32
IMG_SIZE = 224

# Data Augmentation para tornar o modelo mais robusto
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("\n📦 Preparando DataLoaders...")
train_dataset = datasets.ImageFolder(LOCAL_FAST_PATH / 'train', transform=train_transform)
val_dataset = datasets.ImageFolder(LOCAL_FAST_PATH / 'val', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

class_names = train_dataset.classes
print(f"🏷️ Classes detectadas: {class_names}")
print(f"📊 Treino: {len(train_dataset)} imagens")
print(f"📊 Validação: {len(val_dataset)} imagens")

# ==============================================================================
# 4. CRIAR MODELO (MobileNetV3)
# ==============================================================================
print("\n🏗️ Baixando MobileNetV3 (Transfer Learning)...")
model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT)

# Ajustar a última camada para nossas 2 classes (clean vs spotlight)
in_features = model.classifier[3].in_features
model.classifier[3] = nn.Linear(in_features, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 15

print(f"\n🔥 INICIANDO TREINAMENTO ({EPOCHS} épocas)...")
print("💡 DICA: Se precisar parar antes, aperte o botão 'Stop' do Colab.")
print("   O código vai salvar tudo e encerrar suavemente.\n")

best_acc = 0.0
history = {'train_loss': [], 'val_acc': []}

try: # <--- INÍCIO DA PROTEÇÃO
    for epoch in range(EPOCHS):
        # --- TREINO ---
        model.train()
        running_loss = 0.0
        # Usamos leave=False para não poluir a tela com muitas barras
        for inputs, labels in tqdm(train_loader, desc=f"Ep {epoch+1}/{EPOCHS}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # --- VALIDAÇÃO ---
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        epoch_acc = 100 * correct / total
        avg_loss = running_loss / len(train_loader)

        history['train_loss'].append(avg_loss)
        history['val_acc'].append(epoch_acc)

        print(f"   Ep {epoch+1}: Loss: {avg_loss:.4f} | Val Acc: {epoch_acc:.2f}%")

        # 1. SALVAR O MELHOR (BEST)
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(model.state_dict(), OUTPUT_MODEL_DIR / 'spotlight_best.pth') # Mudei nome pra ficar claro
            print("   ✅ Melhor modelo salvo (BEST)!")

        # 2. SALVAR O ÚLTIMO (LAST) - Para poder continuar depois se quiser
        torch.save(model.state_dict(), OUTPUT_MODEL_DIR / 'spotlight_last.pth')

except KeyboardInterrupt:
    print("\n\n🛑 TREINAMENTO INTERROMPIDO PELO USUÁRIO!")
    print("   Salvando progresso atual e finalizando...")

# ==============================================================================
# 6. RESULTADOS FINAIS
# ==============================================================================
print("\n" + "="*50)
print(f"🏆 TREINO FINALIZADO!")
print(f"🎯 Melhor Acurácia Atingida: {best_acc:.2f}%")

# Salvar metadados importantes
config = {
    'model': 'MobileNetV3_Small',
    'classes': class_names,
    'best_acc': best_acc,
    'img_size': IMG_SIZE,
    'last_epoch_reached': len(history['val_acc'])
}

with open(OUTPUT_MODEL_DIR / 'spotlight_config.json', 'w') as f:
    json.dump(config, f, indent=4)

print(f"📂 Modelo BEST salvo em: {OUTPUT_MODEL_DIR}/spotlight_best.pth")
print(f"📂 Modelo LAST salvo em: {OUTPUT_MODEL_DIR}/spotlight_last.pth")
print(f"📂 Config salva em: {OUTPUT_MODEL_DIR}/spotlight_config.json")
print("="*50)

Mounted at /content/drive
🖥️ Hardware selecionado: cuda
🚀 Copiando dataset do Drive para SSD local (Acelerador de Treino)...
   Isso vai levar uns 3-5 minutos, mas vale muito a pena!
✅ Cópia concluída em 25.9 minutos!

📦 Preparando DataLoaders...
🏷️ Classes detectadas: ['clean', 'spotlight']
📊 Treino: 15649 imagens
📊 Validação: 5216 imagens

🏗️ Baixando MobileNetV3 (Transfer Learning)...
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 150MB/s]



🔥 INICIANDO TREINAMENTO (15 épocas)...
💡 DICA: Se precisar parar antes, aperte o botão 'Stop' do Colab.
   O código vai salvar tudo e encerrar suavemente.



Ep 1/15:   0%|          | 0/490 [00:00<?, ?it/s]

   Ep 1: Loss: 0.1174 | Val Acc: 72.12%
   ✅ Melhor modelo salvo (BEST)!


...

   Ep 6: Loss: 0.0221 | Val Acc: 99.67%
   ✅ Melhor modelo salvo (BEST)!


Ep 7/15:   0%|          | 0/490 [00:00<?, ?it/s]

   Ep 7: Loss: 0.0213 | Val Acc: 99.71%
   ✅ Melhor modelo salvo (BEST)!


Ep 8/15:   0%|          | 0/490 [00:00<?, ?it/s]



🛑 TREINAMENTO INTERROMPIDO PELO USUÁRIO!
   Salvando progresso atual e finalizando...

🏆 TREINO FINALIZADO!
🎯 Melhor Acurácia Atingida: 99.71%
📂 Modelo BEST salvo em: /content/drive/MyDrive/document-classifier/models/quality_detector/spotlight_best.pth
📂 Modelo LAST salvo em: /content/drive/MyDrive/document-classifier/models/quality_detector/spotlight_last.pth
📂 Config salva em: /content/drive/MyDrive/document-classifier/models/quality_detector/spotlight_config.json
